In [30]:
import pandas as pd
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
Headers = ['Contig_id', "Nb_genes_contigs", "Fragment", "Nb_genes", "Category", "Nb_Phage_Hallmark", 
           "Phage_gene_enrichment", "Non-Caudovirales_gene_enrichment", "PFAM_Depletion",
           "Uncharactered_Enrichment", "Strand_Switch_Depletion", "Short_Genes"]

#input_file = '../VirSorter/VirSorter_Data/VIRSorter_global-phage-signal_exponential_high.csv'
input_file = '../../Nano_Reads_Tools/VirSorterNanoSimRaw.csv'
commanded = "sed '/^#/d' " + input_file + " > TestingParser.csv"
os.system(commanded)

0

In [32]:
df = pd.read_csv("TestingParser.csv", header=None)
df.columns = Headers
df['Category'] = df['Category'].replace(np.nan, 0)
df['Nb_Phage_Hallmark'] = df['Nb_Phage_Hallmark'].replace(np.nan, 0)
df['Phage_gene_enrichment'] = df['Phage_gene_enrichment'].replace(np.nan, 'None')
df['Category'] = df['Category'].astype(float)

## Remove 
df['Prophage'] = np.where(df['Fragment'].apply(lambda x: 'gene' in x), 1, 0)
df = df[df['Prophage']==0]


df['True_Label'] = [x.split("_")[1] for x in df["Contig_id"]]
df["True_Viral"] = np.where(df['True_Label'].apply(lambda x: 'V' in x), 1, 0)
df.groupby(["Category", "Nb_Phage_Hallmark"]).count()
#df_confidence = df[df['Category']!=3]


df2 = pd.read_csv("../../Contig_Abundance_By_Class_Simulation.csv")
simulation = input_file.split("_")[-2]
complexity = input_file.split("_")[-1].split(".")[0]

df2['Simulation'] = [x.lower() for x in df2['Simulation']]
df2['Complexity'] = [x.lower() for x in df2['Complexity']]

df1 = df2[(df2.Simulation == simulation) & (df2.Complexity == complexity)]
#Phage_Count = int(df1['Phage'])
#Prophage_Count = int(df1['Prophage'])
Phage_Count = 2514

### Without Prophage
df = df[df['Category']!=3]
df = df[df['True_Label']!="Prophage"]
x = df.groupby('True_Label').count()
x = x[['Contig_id']]

Phage_Predicted = int(x.loc["V",])
False_Negative_Phage = Phage_Count - Phage_Predicted


y_true = [1 if x == "V" else 0 for x in df['True_Label']]
y_pred = [1]*len(df)

listofzeros = [0]*False_Negative_Phage
listofones = [1]*False_Negative_Phage

y_pred = y_pred + listofzeros
y_true = y_true + listofones


p = precision_score(y_true, y_pred)
r = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
print("Scores if prophages are considered bacteria")
print("Precision: " + str(round(p,9)))
print("Recall: " + str(round(r,9)))
print("F1 Score: " + str(round(f1,9)))
my_dict = {}
my_dict['Prophage_Bacteria'] = [p,r,f1]
print(round(p,9),round(r,9),round(f1,9))

Scores if prophages are considered bacteria
Precision: 0.911573472
Recall: 0.278838504
F1 Score: 0.427048431
(0.911573472, 0.278838504, 0.427048431)


In [23]:
df2 = pd.read_csv("../../Contig_Abundance_By_Class_Simulation.csv")
simulation = input_file.split("_")[-2]
complexity = input_file.split("_")[-1].split(".")[0]

df2['Simulation'] = [x.lower() for x in df2['Simulation']]
df2['Complexity'] = [x.lower() for x in df2['Complexity']]

df1 = df2[(df2.Simulation == simulation) & (df2.Complexity == complexity)]
Phage_Count = int(df1['Phage'])
Prophage_Count = int(df1['Prophage'])


df = df[df['Category']!=3]
df = df[df['True_Label']!="Prophage"]

x = df.groupby("True_Label").count()
x = x[['Contig_id']]
Phage_Predicted = int(x.loc["Phage",])
False_Negative_Phage = Phage_Count - Phage_Predicted


y_true = [1 if x == "Phage" else 0 for x in df['True_Label']]
y_pred = [1]*len(df)

listofzeros = [0]*False_Negative_Phage
listofones = [1]*False_Negative_Phage

y_pred = y_pred + listofzeros
y_true = y_true + listofones


p = precision_score(y_true, y_pred)
r = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
print("Scores if prophages are considered bacteria")
print("Precision: " + str(round(p,9)))
print("Recall: " + str(round(r,9)))
print("F1 Score: " + str(round(f1,9)))
my_dict = {}
my_dict['Prophage_Bacteria'] = [p,r,f1]
print(round(p,9),round(r,9),round(f1,9))

Scores if prophages are considered bacteria
Precision: 0.833333333
Recall: 0.576923077
F1 Score: 0.681818182
(0.833333333, 0.576923077, 0.681818182)


In [35]:
q = df_confidence.groupby("True_Label").count()
df2 = pd.read_csv("../../Contig_Abundance_By_Class_Simulation.csv")
t = os.path.basename(input_file)

simulation = t.split("_")[-2]
complexity = t.split("_")[-1].split(".")[0]

df1 = df2[(df2.Simulation.str.lower() == simulation) & (df2.Complexity.str.lower() == complexity)]
Phage_Count = int(df1['Phage'])
Prophage_Count = int(df1['Prophage'])






In [38]:
def table_to_output(groupby_table):
    temp = groupby_table
    Phage_Predicted = temp.loc["Phage"][1]
    try:
        Prophage_Predicted = temp.loc["Prophage"][1]
    except:
        Prophage_Predicted = 0
    False_Negative_Phage = Phage_Count - Phage_Predicted
    False_Negative_Prophage = Prophage_Count - Prophage_Predicted
    
    return(False_Negative_Phage, False_Negative_Prophage)

In [39]:
def Score(FNP, FNPro, data, Prophage=True):
    if Prophage==True:
        y_true = [1 if x == "Phage" else 0 for x in data['True_Label']]
        y_pred = [1]*len(data)

        listofzeros = [0]*FNP
        listofones = [1]*FNP

        y_pred = y_pred + listofzeros
        y_true = y_true + listofones

    else:
        data['True_Label'] = data['True_Label'].replace(regex='Prophage', value="Phage")
        y_true = [1 if x == "Phage" else 0 for x in data['True_Label']]
        y_pred = [1]*len(data)
        listofzeros = [0]*FNP
        listofones = [1]*FNP

        y_pred = y_pred + listofzeros
        y_true = y_true + listofones

        listofzeros2 = [0]*FNPro
        listofones2 = [1]*FNPro

        y_pred = y_pred + listofzeros2
        y_true = y_true + listofones2

        
    p = precision_score(y_true, y_pred)
    r = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    return(p, r, f1)


In [43]:
w = pd.DataFrame(my_dict.items(), columns=["Prophage Category",2])

In [41]:
FNP, FNPro = table_to_output(q)
p, r, f1 = Score(FNP, FNPro, df_confidence)

my_dict = {}
print("Scores if prophages are considered Bacteria")
print("Precision: " + str(p))
print("Recall: " + str(r))
print("F1 Score: " + str(f1))
my_dict['Prophage_Bacteria'] = [p,r,f1]

p, r, f1 = Score(FNP, FNPro, df_confidence, Prophage=False)
print("Scores if prophages are considered Viruses")
print("Precision: " + str(p))
print("Recall: " + str(r))
print("F1 Score: " + str(f1))
my_dict['Prophage_Viruses'] = [p,r,f1]

w = pd.DataFrame(my_dict.items(), columns=["Prophage Category",2])
w = w.set_index("Prophage Category")
w['Precision'] = [x[0] for x in w[2]]
w['Recall'] = [x[1] for x in w[2]]
w['F1 Score'] = [x[2] for x in w[2]]
w = w.drop(2, axis=1)

w['Simulation'] = simulation

w['Complexity'] = complexity

w['Tool'] = 'VirSorter'

w['Parameter'] = "NA"
w
#w.to_csv()

Scores if prophages are considered Bacteria
Precision: 0.7588652482269503
Recall: 0.35562941420855837
F1 Score: 0.48429985855728425
Scores if prophages are considered Viruses
Precision: 0.7588652482269503
Recall: 0.024561705546469255
F1 Score: 0.047583312487840125


/Users/stronglab2/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,Precision,Recall,F1 Score,Simulation,Complexity,Tool,Parameter
Prophage Category,,,,,,,
Prophage_Viruses,0.758865,0.024562,0.047583,exponential,high,VirSorter,NA
Prophage_Bacteria,0.758865,0.355629,0.484300,exponential,high,VirSorter,NA


In [44]:
import glob

interesting_files = glob.glob("../VirSorter_Data/*.csv") 

header_saved = False
with open('../VirSorter_Results.csv','wb') as fout:
    for filename in interesting_files:
        with open(filename) as fin:
            header = next(fin)
            if not header_saved:
                fout.write(header)
                header_saved = True
            for line in fin:
                fout.write(line)